# BioInformatics Assignment 3
## Matthew Johnson, Oct. 27, 2018

In [185]:
# File with original DNA sequences in FASTA format
filename = 'vertebrates.txt'

with open(filename) as f:
    data = f.readlines()
    
len(data)

414

In [186]:
import re
    '''
    dna_seq: class that takes in FASTA format string and creates an object with a name, sequence, raw, and new_raw 
    attributes
    '''
class dna_seq:
    def __init__(self, raw):
        self.name = raw.split('\n')[0]
        self.sequence = ''.join(raw.split('\n')[1:]).replace(' ', '')
        self.raw = '>' + raw
        self.new_raw = ''
        
    def new_sequence(self, pattern):
        # n - the index of the pattern if found, else -1
        n = self.sequence.find(pattern)
        if n != -1:
            print(f'Sequence Shifted, index = {n}')
            self.shifted_sequence = self.sequence[n:] + self.sequence[:n]
        else:
            print(f"Whoops! Couldn't find that pattern in {self.name}")
            self.shifted_sequence = self.sequence
        # regex to insert newline every 60 characters in sequence (CLUSTALW won't work without this)
        self.new_raw = ''.join(['>', self.name, re.sub("(.{60})", "\\1\n", self.shifted_sequence, 0, re.DOTALL)])
        return self

### 1. Re-arrange the 8 circular mitochondrion DNA provided in the given file so that the highly reserved areas appear at the beginning of all the sequences.

In [187]:
# Join the array of sequence data and split by '>, don't take the first element (its blank)
sequences = ' '.join(data).split('>')[1:]

# Looked like a decent enough pattern? Not sure
pattern = 'CAAACTGGGATTAGATACCCCACTATGC'

# Create a dna_seq object for each sequence, divides out the name and sequenec from raw string
seq_list = [dna_seq(seq) for seq in sequences]

# Create new sequences by searching for the pattern and reindexing the sequence to the position of the pattern start
new_seqs = [seq.new_sequence(pattern).new_raw.rstrip('\n') for seq in seq_list]

Sequence Shifted, index = 1065
Sequence Shifted, index = 483
Sequence Shifted, index = 484
Sequence Shifted, index = 841
Sequence Shifted, index = 1396
Sequence Shifted, index = 914
Sequence Shifted, index = 1412
Sequence Shifted, index = 1727


In [190]:
# Writing filename
filename2 = 'vertebrates3.txt'

with open(filename2, 'w') as f:
    f.write('\n \n'.join(new_seqs))

### 2. Perform multiple sequence alignments using Clustal-omega.for both the original file and the file that has been rearranged.
- Completed

### 3. Compare the two multiple alignments and elaborate what you have noticed and learned.
   - Old = 4900 cols 
   - New = ~3200
   - *What else..?*
   
### 4. Compare the two Percent Identity Matrices and elaborate what you have noticed and learned.
- Numbers are lower?

In [198]:
# Percent Identity Matrix (Before alignment)
with open('pim1.txt') as f:
    text = f.readlines()
    
# Percent Identity Matrix (After alignment)
with open('pim2.txt') as f:
    text2 = f.readlines()
    
    
animals, animals2 = [], []
numbers, numbers2 = [], []

for i in range(8):
    animals.append(text[i].strip().rstrip('\n').split()[1])
    numbers.append(text[i].strip().rstrip('\n').split()[2:])
    
    animals2.append(text2[i].rstrip('\n').strip().split()[1])
    numbers2.append(text2[i].rstrip('\n').strip().split()[2:])
    
df1 = pd.DataFrame(numbers, columns=animals, index=animals).astype(float)
df2 = pd.DataFrame(numbers2, columns=animals2, index=animals2).astype(float)
df2 = df2.reindex(index = df1.columns)[df1.columns]
df3 = df2 - df1  # df1 >> df2

## Percent Identity Matrices

In [200]:
df1 # BEFORE

,chicken,carp,human,whale,seal,cow,mouse,rat
chicken,100.00,54.01,53.68,57.18,54.04,57.47,59.35,58.86
carp,54.01,100.00,56.08,57.37,55.81,58.77,61.74,61.11
human,53.68,56.08,100.00,65.86,64.75,66.22,67.58,67.79
whale,57.18,57.37,65.86,100.00,67.88,74.63,71.83,68.97
seal,54.04,55.81,64.75,67.88,100.00,70.97,72.19,72.15
cow,57.47,58.77,66.22,74.63,70.97,100.00,72.97,71.33
mouse,59.35,61.74,67.58,71.83,72.19,72.97,100.00,82.37
rat,58.86,61.11,67.79,68.97,72.15,71.33,82.37,100.00


In [201]:
df2 # AFTER

,chicken,carp,human,whale,seal,cow,mouse,rat
chicken,100.00,51.16,51.38,50.84,50.63,50.71,50.35,49.81
carp,51.16,100.00,52.78,54.47,53.70,55.60,53.97,54.34
human,51.38,52.78,100.00,63.10,60.78,63.46,61.76,61.18
whale,50.84,54.47,63.10,100.00,62.24,71.70,65.81,63.34
seal,50.63,53.70,60.78,62.24,100.00,65.23,60.99,61.56
cow,50.71,55.60,63.46,71.70,65.23,100.00,68.61,67.99
mouse,50.35,53.97,61.76,65.81,60.99,68.61,100.00,78.54
rat,49.81,54.34,61.18,63.34,61.56,67.99,78.54,100.00


In [199]:
df3 # NET CHANGE

,chicken,carp,human,whale,seal,cow,mouse,rat
chicken,0.00,-2.85,-2.30,-6.34,-3.41,-6.76,-9.00,-9.05
carp,-2.85,0.00,-3.30,-2.90,-2.11,-3.17,-7.77,-6.77
human,-2.30,-3.30,0.00,-2.76,-3.97,-2.76,-5.82,-6.61
whale,-6.34,-2.90,-2.76,0.00,-5.64,-2.93,-6.02,-5.63
seal,-3.41,-2.11,-3.97,-5.64,0.00,-5.74,-11.20,-10.59
cow,-6.76,-3.17,-2.76,-2.93,-5.74,0.00,-4.36,-3.34
mouse,-9.00,-7.77,-5.82,-6.02,-11.20,-4.36,0.00,-3.83
rat,-9.05,-6.77,-6.61,-5.63,-10.59,-3.34,-3.83,0.00


1. Put the
* original file, 
* the re-arranged file,
*  the two alignment files (CLUSTAL format with base numbering), 
* the two files for Percent Identity Matrices, 
* the file contains your analysis,
    * Comparing the two multiple alignments
    * Comparing the two percent identity matrices
   into a folder, ZIP the folder, and submit the ZIP file through Moodle.